In [2]:
import numpy as np
import pandas as pd

In [36]:
#Read passengers
passengers = pd.read_csv('passengers.csv')

passengers['country_code'] = passengers['country_code'].replace({'EL': 'GR'})
passengers.head()

,direction,coverage,year,passengers,country_code,airport_code
0,Arrivals,International,2017,408716,AT,LOWG
1,Arrivals,International,2018,441657,AT,LOWG
2,Arrivals,International,2019,448662,AT,LOWG
3,Arrivals,International,2020,76334,AT,LOWG
4,Arrivals,International,2021,82843,AT,LOWG


In [37]:
#Airport locations
locations = pd.read_csv('airports_locations.csv')
locations.head()

,icao,airport,latitude,longitude
0,LOWL,Linz Airport (Blue Danube Airport),48.2332,14.1875
1,LOWS,Salzburg Airport,47.7933,13.0043
2,LOWK,Klagenfurt Airport,46.6425,14.3377
3,LOWG,Graz Airport,46.9911,15.4396
4,LOWI,Innsbruck Airport (Kranebitten Airport),47.2602,11.3440


In [38]:
#Countries names
countries = pd.read_json('CountryList.json')
countries.head()

,name,code,region
0,Albania,AL,Central and Eastern Europe
1,Andorra,AD,Western Europe
2,Argentina,AR,South America
3,Armenia,AM,Central and Eastern Europe
4,Austria,AT,Western Europe


In [43]:
countries[countries['code'] == 'GR']

,name,code,region
31,Greece,GR,Southern Europe


In [55]:
country_2022 = passengers[passengers['year'] == 2022].groupby(['country_code'])['passengers'].sum().reset_index()
country_2019 = passengers[passengers['year'] == 2019].groupby(['country_code'])['passengers'].sum().reset_index()
result = pd.merge(country_2019, country_2022, on='country_code', how='inner', suffixes=('_2019', '_2022'))
result['resta'] = (result['passengers_2022'] - result['passengers_2019'])/result['passengers_2019'] * 100


In [58]:
result = pd.merge(result, countries, left_on='country_code', right_on='code', how='inner')

result = result.drop(columns = ['country_code','code','region','passengers_2019','passengers_2022']).sort_values(by='resta', ascending=False)

In [59]:
result.to_csv('variacio_paisos.csv', index=False)

In [72]:
airport_2022 = passengers[passengers['year'] == 2022].groupby(['airport_code'])['passengers'].sum().reset_index()
airport_2019 = passengers[passengers['year'] == 2019].groupby(['airport_code'])['passengers'].sum().reset_index()
result = pd.merge(airport_2019, airport_2022, on='airport_code', how='inner', suffixes=('_2019', '_2022'))
result['resta'] = (result['passengers_2022'] - result['passengers_2019'])/result['passengers_2019'] * 100

In [73]:
result = pd.merge(result, locations, left_on='airport_code', right_on='icao', how='inner')
result = result.drop(columns = ['airport_code','icao','passengers_2022',]).sort_values(by='passengers_2019', ascending=False)

In [77]:
result['group'] = result['resta'].apply(lambda x: 'Positive' if x > 0 else 'Negative')

In [81]:
result.to_csv('variacio_aeroports.csv', index=False)

In [80]:
result.head()

,passengers_2019,resta,airport,latitude,longitude,group
197,76195486,-24.549387,Paris Charles de Gaulle Airport,49.0097,2.54778,Negative
50,71742574,-26.859428,Amsterdam Airport Schiphol,52.3086,4.76389,Negative
12,70634781,-30.778765,Frankfurt Airport,50.0333,8.57056,Negative
161,59816612,-15.938684,Adolfo Suarez Madrid-Barajas Airport,40.4719,-3.56264,Negative
374,52031566,24.024904,Istanbul Ataturk Airport,41.2753,28.75190,Positive


In [92]:
result = passengers.groupby(['year','country_code'])['passengers'].sum().reset_index()

result = pd.merge(result, countries, left_on='country_code', right_on='code', how='inner')
result = result.drop(columns = ['country_code','code','region'])
result = result.pivot(index='year', columns='name', values='passengers')

In [93]:
result.to_csv('country_years.csv', index=True)